## Introduction to Databases

### SQLAlchemy 

In [32]:
#!pip install mysqlclient
#!pip install python-dotenv

In [33]:
import os
import getpass

from sqlalchemy import create_engine, Column, Integer, String, Sequence, Float, PrimaryKeyConstraint, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, backref
#from sqlalchemy.sql import *
from dotenv import load_dotenv, find_dotenv

In [34]:
p = getpass.getpass()

········


In [35]:
def connect_db():
    db = create_engine('mysql://root:' + p + '@localhost/mysql')
    return db

In [36]:
Base = declarative_base()

Criação das Tabelas

In [37]:
class Edificio(Base):
    __tablename__ = "edificio"
    
    id_edificio = Column(Integer, Sequence('id_edificio_seq'), primary_key = True)
    nome = Column(String(100), index=True, nullable = True)
    ano_de_construcao = Column(String(100), index = True, nullable = True)
    endereco = Column(String(100), index = True, nullable = True)

In [38]:
estilo = relationship('Estilo', backref = 'edificio',lazy=True, cascade = "all, delete-orphan")
arquiteto = relationship('Arquiteto', backref = 'edificio', lazy = True, cascade = "all, delete-orphan")
categoria = relationship('CategoriaEdificio', backref = 'edificio', lazy = True,cascade = "all, delete-orphan")

In [39]:
class Estilo(Base):
    __tablename__="estilo"
    __table_args__ = (
        PrimaryKeyConstraint('id_edificio', 'estilo'),
    )
    id_edificio =Column(Integer,ForeignKey('edificio.id_edificio'))
    estilo = Column(String(100), index = True)

class Arquiteto(Base):
    __tablename__="arquiteto"
    __table_args__ = (
        PrimaryKeyConstraint('id_edificio', 'nome_arquiteto'),
    )
    id_edificio= Column(Integer,ForeignKey('edificio.id_edificio'))
    nome_arquiteto = Column(String(100), index = True)

class CategoriaEdificio(Base):
    __tablename__="categoria_edificio"
    __table_args__ = (
        PrimaryKeyConstraint('id_edificio', 'categoria'),
    )
    id_edificio = Column(Integer,ForeignKey('edificio.id_edificio'))
    categoria = Column(String(100), index = True)

In [40]:
engine = connect_db()
Edificio.__table__.create(bind=engine, checkfirst=True)
Estilo.__table__.create(bind=engine, checkfirst=True)
Arquiteto.__table__.create(bind=engine, checkfirst=True)
CategoriaEdificio.__table__.create(bind=engine, checkfirst=True)

In [41]:
db = connect_db()
Session = sessionmaker(bind=db)
session = Session()

Inserção dos Dados

In [42]:
session.add(Edificio(nome = 'Linked Hybrid', ano_de_construcao = '2009', endereco = 'Pequim - China'))
session.add(Edificio(nome = 'Casa Moby Dick', ano_de_construcao = '2003', endereco = 'Espoo - Finlândia'))
session.add(Edificio(nome = 'Apartamentos WoZoCo', ano_de_construcao = '1997', endereco = 'Amsterdã - Holanda'))
session.add(Edificio(nome = 'Casas Cubo', ano_de_construcao = '1984', endereco = 'Roterdã - Holanda'))
session.add(Edificio(nome = 'Museu Perot de Ciência Natural', ano_de_construcao = '2012', endereco = 'Dallas - Estados Unidos'))
session.commit()

In [43]:
session.add(CategoriaEdificio(id_edificio = 1, categoria = "Comércio"))
session.add(CategoriaEdificio(id_edificio = 2, categoria = "Exposição"))
session.add(CategoriaEdificio(id_edificio = 3, categoria = "Moradia"))
session.add(CategoriaEdificio(id_edificio = 4, categoria = "Comércio"))
session.add(CategoriaEdificio(id_edificio = 5, categoria = "Museu"))
session.commit()

In [44]:
session.add(Arquiteto(id_edificio = 1, nome_arquiteto = "Su Dong Xia"))
session.add(Arquiteto(id_edificio = 2, nome_arquiteto = "Paavo Seppo Aadolf"))
session.add(Arquiteto(id_edificio = 3, nome_arquiteto = "Norbert Ágoston Odd"))
session.add(Arquiteto(id_edificio = 4, nome_arquiteto = "Jere Petri Aulis"))
session.add(Arquiteto(id_edificio = 5, nome_arquiteto = "Jóhannes Amrit Bogdan"))
session.commit()

Exploração

In [45]:
session.query(Edificio).count()

10

In [46]:
deletar_arquiteto = session.query(Arquiteto).filter(Arquiteto.id_edificio==1).first()
session.delete(deletar_arquiteto)
session.commit()

In [47]:
import pandas as pd
df = pd.read_sql(session.query(Edificio).statement, session.bind)

In [48]:
df

,id_edificio,nome,ano_de_construcao,endereco
0,1,Linked Hybrid,2009,Pequim - China
1,2,Casa Moby Dick,2003,Espoo - Finlândia
2,3,Apartamentos WoZoCo,1997,Amsterdã - Holanda
3,4,Casas Cubo,1984,Roterdã - Holanda
4,5,Museu Perot de Ciência Natural,2012,Dallas - Estados Unidos
5,6,Linked Hybrid,2009,Pequim - China
6,7,Casa Moby Dick,2003,Espoo - Finlândia
7,8,Apartamentos WoZoCo,1997,Amsterdã - Holanda
8,9,Casas Cubo,1984,Roterdã - Holanda
9,10,Museu Perot de Ciência Natural,2012,Dallas - Estados Unidos


In [49]:
df.query("ano_de_construcao >= '2000'")

,id_edificio,nome,ano_de_construcao,endereco
0,1,Linked Hybrid,2009,Pequim - China
1,2,Casa Moby Dick,2003,Espoo - Finlândia
4,5,Museu Perot de Ciência Natural,2012,Dallas - Estados Unidos
5,6,Linked Hybrid,2009,Pequim - China
6,7,Casa Moby Dick,2003,Espoo - Finlândia
9,10,Museu Perot de Ciência Natural,2012,Dallas - Estados Unidos


In [50]:
df.query('endereco.str.contains("Holanda")', engine='python')

,id_edificio,nome,ano_de_construcao,endereco
2,3,Apartamentos WoZoCo,1997,Amsterdã - Holanda
3,4,Casas Cubo,1984,Roterdã - Holanda
7,8,Apartamentos WoZoCo,1997,Amsterdã - Holanda
8,9,Casas Cubo,1984,Roterdã - Holanda
